# Datos Lluvia WRF 1.5 Ajustados con Climatología

## Cargar Librerias
Se cargan las librerías esenciales para la manipulación de datos, visualización, cálculos numéricos, manejo de fechas, interacción con la web y operaciones del sistema operativo.

In [1]:
import glob # Para buscar archivos en un directorio.
import pandas as pd # Para manipulación de datos.
import matplotlib.pyplot as plt # Para visualización de datos en 2D.
import numpy as np # Para cálculos numéricos.
from datetime import datetime, date, timedelta # Para fechas y horas.
import requests # Para realizar solicitudes HTTP.
import urllib.request # Para trabajar con URLs.
import io # Para manejar entradas/salidas.
import os # Para interactuar con el sistema operativo.

## Descarga de datos desde el servidor para un rango de fechas

La función `descargar_datos_desde_servidor` recibe dos parámetros: `enlace_base` que señala la ubicación de los archivos en el servidor, y `fecha_inicio` que marca el punto de partida temporal. La función inicia obteniendo la fecha actual más un día para establecer la fecha final de descarga. Luego, inicializa dos listas vacías: `dataframes` para almacenar los DataFrames (datos) y `file_name` para las fechas de archivo correspondientes. 

Se itera a través del rango de fechas desde la fecha de inicio hasta la fecha final. En cada iteración, forma el enlace completo para descargar el archivo CSV correspondiente a cada día en el rango. Se realiza una solicitud HTTP GET para obtener el archivo CSV del servidor, y si la solicitud es exitosa (código de estado 200), los datos se decodifican y se crea un DataFrame de Pandas. 

Se realiza un cálculo para sumar las columnas 1, 2, 3 y 4 del DataFrame, y se agrega una nueva columna llamada "Valor" con el resultado. El DataFrame resultante se agrega a la lista de DataFrames, y el nombre de archivo correspondiente (fecha siguiente) se agrega a la lista de nombres de archivo.

Finalmente, crea una nueva lista llamada `data_modelo` que contiene subconjuntos de cada DataFrame original con solo las columnas "Estacion" y "Valor". La función devuelve una tupla con `data_modelo` y `file_name`.

In [2]:
def descargar_datos_desde_servidor(enlace_base, fecha_inicio, fecha_fin):
    """
    Función que carga todos los archivos de lluvia pronosticada desde el servidor
    
    Parameters:
        enlace_base (str): Enlace base del servidor donde se encuentran los archivos de lluvia del modelo.
        fecha_inicio (datetime.date): Fecha en la que se iniciará la descarga de los archivos.
        fecha_fin (datetime.date): Fecha en la que se finalizará la descarga de los archivos.

    Returns:
        Una tupla que contiene dos elementos:
            - data_modelo: Una lista de DataFrames con los datos de lluvia del modelo para cada día del rango.
            - file_date: Una lista de fechas de archivo correspondientes a cada DataFrame descargado en data_modelo.
    """

    # Inicializar listas vacías para almacenar los DataFrames de lluvia pronosticada y los fechas correspondientes.
    dataframes, file_date = [], []

    fecha_actual = fecha_inicio # Establecer fecha actual como fecha de incio

    # Iterar sobre el rango de fechas
    while fecha_actual < fecha_fin:
        fecha_siguiente = fecha_actual + timedelta(days=1) # Obtener la fecha siguiente

        # Formar el enlace completo para el archivo CSV con la fecha actual y siguiente
        enlace_siguiente = f"{enlace_base}{fecha_actual.strftime('%Y%m%d')}/{fecha_siguiente.strftime('%Y%m%d')}.lluvia.csv"

        # Realizar la solicitud HTTP GET para obtener el archivo CSV
        response = requests.get(enlace_siguiente)

        if response.status_code == 200: # Verificar si la solicitud fue exitosa (código de estado 200)
            csv_data = response.content.decode('utf-8') 
            # Obtener los datos CSV de la respuesta y decodificarlos como texto UTF-8

            # Crear un DataFrame de Pandas a partir del archivo CSV y especificar el encabezado de columnas
            df = pd.read_csv(io.StringIO(csv_data), header=None, names=["Estacion", "1", "2", "3", "4"])
            
            # Verificar si las columnas "Tmax" y "Tmin" solo contienen valores str
            if df['1'].apply(lambda x: isinstance(x, str)).all() and df['2'].apply(lambda x: isinstance(x, str)).all() and df['3'].apply(lambda x: isinstance(x, str)).all() and df['4'].apply(lambda x: isinstance(x, str)).all():
                print(f"Valores en Tmax y Tmin para la fecha {fecha_siguiente} son del tipo str. No se carga el archivo.")
            else:
                # Sustituir los valores str por NaN en las columnas
                df[["1", "2", "3", "4"]] = df[["1", "2", "3", "4"]].apply(pd.to_numeric, errors='coerce')
                # Imprimir una nota sobre los valores sustituidos por NaN
                replaced_values = df[df.applymap(lambda x: isinstance(x, str))].dropna(how='all')
                
                #Imprimir mensaje si hay valores reemplazados
                if not replaced_values.empty:
                    print(f"Valores sustituidos por NaN para la fecha {fecha_siguiente}:\n{replaced_values}")
            
                # Calcular la suma de las columnas 1, 2, 3 y 4 y almacenar el resultado en la columna 'Valor'
                df['Valor'] = df.iloc[:, 1] + df.iloc[:, 2] + df.iloc[:, 3] + df.iloc[:, 4]

                dataframes.append(df) # Agregar el DataFrame de lluvia pronosticada a la lista de DataFrames

                file_date.append(fecha_siguiente.strftime('%Y%m%d')) # Agregar la fecha de archivo correspondiente
        else:
            #Imprimir un mensaje si el archivo de lluvia pronosticada no se encontró
            print(f"El archivo para la fecha {fecha_siguiente} no se encontró en el servidor.")

        fecha_actual = fecha_siguiente # Actualizar la fecha actual para la próxima iteración del ciclo
    
    # Crear una lista de DataFrames filtrados para el modelo
    data_modelo = [tabla[['Estacion', 'Valor']] for tabla in dataframes] 
    return data_modelo, file_date

### Descarga de datos desde el servidor para un día

La función `obtener_datos_un_dia` permite obtener y procesar los datos de precipitación para un día desde un enlace. Recibe como parámetro `enlace_base`, que es la ruta base donde se encuentra el archivo de datos. La función obtiene la fecha actual y la fecha siguiente en formato "YYYYMMDD". Luego, construye el enlace completo para el archivo CSV utilizando estas fechas y el enlace base.

Después, se realiza una solicitud HTTP GET para obtener el archivo CSV del servidor, y si la solicitud es exitosa (código de estado 200), los datos se decodifican, se crea un DataFrame de Pandas y se agrega un encabezado personalizado para las columnas. Los datos contienen información sobre estaciones y cuatro columnas de precipitación, y se realiza un cálculo para sumar las columnas 1, 2, 3 y 4 del DataFrame para obtener una columna llamada "Valor".

Finalmente, se seleccionan solo las columnas "Estacion" y "Valor" para crear una nueva tabla llamada `data_modelo` que contiene la información relevante. La función devuelve una tupla que contiene `data_modelo` y `file_date`.

In [3]:
def obtener_datos_un_dia(enlace_base):
    """
    Función que carga el archivo correspondiente a la lluvia pronosticada del día siguiente desde el servidor
    
    Parameters:
        enlace_base (str): Enlace base del servidor donde se encuentran los archivos.

    Returns:
        tuple: Una tupla que contiene dos elementos:
            - data_modelo: Un DataFrame con los datos para el día siguiente.
            - file_date: La fecha del archivo correspondiente al día siguiente.
    """

    try:
        # Obtener la fecha actual y la fecha siguiente en formato "YYYYMMDD"
        fecha_actual = date.today().strftime("%Y%m%d")
        fecha_siguiente = (date.today() + timedelta(days=1)).strftime("%Y%m%d")

        # Formar el enlace completo para el archivo de lluvia pronosticada con la fecha actual y siguiente
        enlace_completo = f"{enlace_base}{fecha_actual}/{fecha_siguiente}.lluvia.csv"

        # Realizar la solicitud HTTP GET para obtener el archivo CSV
        response = requests.get(enlace_completo)

        if response.status_code == 200: # Verificar si la solicitud fue exitosa (código de estado 200)
            # Leer los datos directamente desde la respuesta y agregar un encabezado personalizado
            encabezado = ["Estacion", "1", "2", "3", "4"]
            data = pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep=",", header=None, names=encabezado)

            # Se suman las 4 columnas de lluvia pronosticada en cada DataFrame y se crea una nueva columna
            # Verificar si las columnas contienen solo valores str
            if data['1'].apply(lambda x: isinstance(x, str)).all() and data['2'].apply(lambda x: isinstance(x, str)).all() and data['3'].apply(lambda x: isinstance(x, str)).all() and data['4'].apply(lambda x: isinstance(x, str)).all():
                print(f"Valores de lluvia pronosticada para la fecha {fecha_siguiente} son del tipo str. No se carga el archivo.")
            else:
                # Sustituir los valores str por NaN
                data[["1", "2", "3", "4"]] = data[["1", "2", "3", "4"]].apply(pd.to_numeric, errors='coerce')
                # Imprimir una nota sobre los valores sustituidos por NaN
                replaced_values = data[data.applymap(lambda x: isinstance(x, str))].dropna(how='all')
                
                #Imprimir mensaje si hay valores reemplazados
                if not replaced_values.empty:
                    print(f"Valores sustituidos por NaN para la fecha {fecha_siguiente}:\n{replaced_values}")

                 # Calcular la suma de las columnas 1, 2, 3 y 4 y almacenar el resultado en la columna 'Valor'
                data['Valor'] = data.iloc[:, 1] + data.iloc[:, 2] + data.iloc[:, 3] + data.iloc[:, 4]

        # Crear un DataFrames filtrado
        data_modelo = data[["Estacion", "Valor"]]
        file_date = fecha_siguiente # Guardar la fecha en una nueva variable

        return data_modelo, file_date
    
    # Mensaje de error si no se obtienen archivos
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener el archivo: {e}")
        return None, None

### Carga de Tablas Climatología Mensual (Observada)

La función `cargar_tablas_desde_excel` se encarga de cargar las tablas desde un archivo Excel y convertir los nombres de las columnas a cadenas de texto.

La función devuelve un diccionario que contiene las tablas cargadas. Las claves de este diccionario son los nombres de las hojas de Excel proporcionadas en la lista `hojas_excel`, y los valores son DataFrames de Pandas que representan las tablas con los datos.

El proceso que sigue la función es iterar sobre cada nombre de hoja en la lista `hojas_excel`. Para cada hoja, carga los datos desde el archivo Excel utilizando Pandas. Luego, convierte los nombres de las columnas a cadenas de texto. Posteriormente, agrega la tabla a un diccionario llamado `tablas`, donde la clave es el nombre de la hoja y el valor es el DataFrame que contiene los datos de esa hoja.

Finalmente, la función devuelve el diccionario `tablas` que contiene todas las tablas cargadas desde el archivo Excel, cada una identificada por el nombre de la hoja correspondiente.

In [4]:
def cargar_tablas_desde_excel(archivo_excel, hojas_excel):
    """
    Cargar las tablas de la climatología mensual de la lluvia observada
    
    Parameters:
        archivo_excel (str): Ruta del archivo Excel que contiene las tablas.
        hojas_excel (list): Lista de nombres de las hojas de Excel a cargar.

    Returns:
        dict: Un diccionario que contiene las tablas cargadas. Las claves son los nombres de las hojas y los valores son DataFrames de Pandas.
    """
    # Crear un diccionario para almacenar las tablas de la climatología mensual (lluvia observada) cargadas desde Excel
    tablas = {}

    # Iterar sobre la lista de nombres de hojas de Excel a cargar
    for hoja in hojas_excel:
        # Cargar la tabla desde la hoja de Excel especificada
        tabla = pd.read_excel(archivo_excel, sheet_name=hoja)
        
        # Convertir los nombres de las columnas a cadenas de texto
        tabla.columns = tabla.columns.astype(str)
        
        # Agregar la tabla al diccionario utilizando el nombre de la hoja como clave
        tablas[hoja] = tabla

    return tablas

### Ajuste de Datos (Rango de Fechas) con Climatología Mensual (Observada)

La función `procesar_datos_all` realiza el procesamiento y ajuste de los datos descargados, teniendo en cuenta los valores mínimos y máximos definidos en las tablas de climatología proporcionadas. La función toma los siguientes parámetros: `data_modelo`, `file_date`,`tablas` y `carpeta`, y devuelve `tabla_ajuste_lluvia` como resultado.

La función crea un nuevo DataFrame llamado `tabla_ajuste_lluvia` que tendrá una columna "FECHA" y columnas adicionales para cada estación presente en los DataFrames cargados del modelo. Luego, itera sobre cada DataFrame y fecha correspondiente, procesando los valores y ajustándolos según los límites definidos en las tablas.

Para cada estación y fecha, se verifica si el valor de precipitación cumple con los límites establecidos en las tablas `min` y `max` de la climatología. Si el valor excede el máximo, se reemplaza con el valor máximo y se almacena en la nueva tabla con una notación indicando el cambio. Si el valor es menor que cero, se reemplaza con el valor mínimo y se registra en la tabla de ajuste. Si el valor se encuentra dentro de los límites, se marca como "-" en la tabla de ajuste. Si no hay datos para una estación en una fecha determinada, se marca como "x" en la tabla de ajuste.

Luego, se guarda la `tabla_ajuste_lluvia` y cada DataFrame ajustado.

In [5]:
def procesar_datos_all(data_modelo, file_date, tablas, carpeta):
    """
    Ajustar los datos de lluvia pronosticada con las tablas de la climatología mensual de lluvia observada
    
    Parameters:
        data_modelo (list): Lista de DataFrames que contienen los datos de lluvia pronosticada.
        file_date (list): Lista de fechas de archivo correspondientes a cada DataFrame en data_modelo.
        tablas (dict): Diccionario con las tablas cargadas de climatología mensual (lluvia observada) desde Excel.
        carpeta (str): Ruta donde se guardarán los archivos de resultados.

    Returns:
        tabla_ajuste_lluvia: Un DataFrame con los datos procesados y ajustados.
    """

    # Crear una tabla para el ajuste de lluvia con las columnas adecuadas
    tabla_ajuste_lluvia = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_modelo[0]['Estacion']])
    
    # Iterar sobre los DataFrames de lluvia pronosticada y las fechas correspondientes
    for tabla, fecha in zip(data_modelo, file_date):
        fecha_dt = datetime.strptime(fecha, "%Y%m%d")  # Convertir fecha a formato de datetime
        mes = fecha_dt.month  # Obtener el mes correspondiente

        nueva_fila = pd.DataFrame({"FECHA": [fecha_dt.strftime("%Y%m%d")]})  # Crear una nueva fila con la fecha correspondiente

        # Iterar sobre las estaciones en la tabla_ajuste_lluvia
        for estacion in tabla_ajuste_lluvia.columns[1:]:
            estacion = str(estacion)

            # Obtener el valor de climatología mensual de lluvia observada para la estación y el mes correspondiente
            valor_min = tablas['min'].loc[tablas['min']["FECHA"] == mes, estacion].values[0]
            valor_max = tablas['max'].loc[tablas['max']["FECHA"] == mes, estacion].values[0]

            # Filtrar las filas de la lluvia pronosticada por estación
            filas_estacion = tabla.loc[tabla["Estacion"].astype(str).str.strip() == estacion]

            # Verificar que la fila para la 'estación' en lluvia pronosticada no esté vacía
            if not filas_estacion.empty:
                # Obtener el valor de la lluvia pronosticada para la estación y el mes
                valor_modelo = filas_estacion["Valor"].iloc[0]

                # Ajustar la lluvia pronosticada con los limites de la climatología mensual de lluvia observada
                if valor_modelo >= valor_max:
                    nueva_fila[estacion] = f"{valor_max}({valor_modelo})"
                    tabla.loc[filas_estacion.index, "Valor"] = valor_max
                elif valor_modelo < 0:
                    nueva_fila[estacion] = f"{valor_min}({valor_modelo})"
                    tabla.loc[filas_estacion.index, "Valor"] = valor_min
                else:
                    nueva_fila[estacion] = "-" # Si el valor para la estación está dentro de los limites, asignar "-"
            else:
                nueva_fila[estacion] = "x"  # Si no hay valor para la estación en la lluvia pronosticada, asignar "x"

        #Agregar la nueva fila a la tabla de ajuste
        tabla_ajuste_lluvia = pd.concat([tabla_ajuste_lluvia, nueva_fila], ignore_index=True)

        # Guardar cada dataframe de data_modelo (lluvia pronosticada) en formato CSV y TXT
        tabla.to_csv(f"{carpeta}/{fecha}.lluvia.csv", index=False)
        tabla.to_csv(os.path.join(carpeta, f"{fecha}.lluvia.txt"), index=False)

    # Guardar la tabla de ajuste en CSV y TXT
    tabla_ajuste_lluvia.to_csv(f"{carpeta}/tabla_ajuste_lluvia.csv", index=False)
    tabla_ajuste_lluvia.to_csv(os.path.join(carpeta, "tabla_ajuste_lluvia.txt"), index=False)
    
    return tabla_ajuste_lluvia

### Ajuste de Datos (Un Día) con Climatología Mensual (Observada)

La función `procesar_datos_un_dia` se encarga de procesar los datos de precipitación para un día actual y agregar una nueva fila con las estaciones como columnas. Los parámetros de la función son los siguientes:`tabla_ajuste_lluvia`, `data_modelo`, `file_date`, `tablas` y `carpeta`, y la función devuelve la `tabla_ajuste_lluvia`.

La función comienza creando una nueva fila con la fecha siguiente al día actual y las estaciones como columnas. Luego, verifica si la fecha siguiente ya existe en la tabla `tabla_ajuste_lluvia`. Si la fecha existe, se elimina la fila correspondiente para reemplazarla posteriormente con los nuevos datos.

Después, se crea una nueva fila con la fecha actual y se inicializa con valores vacíos para cada estación. Se itera sobre cada estación y se verifica si el valor de precipitación cumple con los límites establecidos en las tablas `min` y `max` de la climatología. Si el valor excede el máximo, se reemplaza con el valor máximo y se almacena en la tabla de ajuste con una notación indicando el cambio. Si el valor es menor que cero, se reemplaza con el valor mínimo y se registra en la tabla de ajuste. Si el valor se encuentra dentro de los límites, se marca como "-". Si no hay filas que coincidan con la estación en `data_modelo`, se asigna "x" a la nueva fila en la columna correspondiente.

Una vez completada la nueva fila con los valores procesados, se agrega al DataFrame `tabla_ajuste_lluvia`. Posteriormente, se guarda la tabla de ajuste en un archivo CSV y los datos descargados para el día siguiente se guardan en un archivo CSV en la carpeta especificada.

In [6]:
def procesar_datos_un_dia(tabla_ajuste_lluvia, data_modelo, file_date, tablas, carpeta):
    """
    Ajustar los datos de lluvia pronosticada con las tablas de la climatología mensual de lluvia observada
    
    Parameters:
        tabla_ajuste_lluvia (pd.DataFrame): DataFrame que contiene los datos procesados hasta el momento.
        data_modelo (pd.DataFrame): DataFrame con los datos para el día siguiente.
        file_date (str): Nombre del archivo que contiene los datos para el día siguiente.
        tablas (dict): Diccionario con las tablas cargadas desde un archivo Excel.
        carpeta (str): Ruta de la carpeta donde se guardarán los archivos CSV.

    Returns:
        pd.DataFrame: Un DataFrame con los datos procesados y ajustados.
    """

    # Obtener la fecha en formato texto (str)
    fecha_siguiente = (date.today() + timedelta(days=1))
    fecha_siguiente_str = fecha_siguiente.strftime("%Y%m%d")

    # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
    fecha_siguiente_int = np.int64(fecha_siguiente_str)

    # Verificar si la fecha ya existe en la tabla
    if fecha_siguiente_int in tabla_ajuste_lluvia["FECHA"].values:
        # Si la fecha ya existe, obtener el índice de la fila existente
        idx_fecha_existente = tabla_ajuste_lluvia.index[tabla_ajuste_lluvia["FECHA"] == fecha_siguiente_int][0]
        
        # Eliminar la fila existente a partir del índice
        tabla_ajuste_lluvia = tabla_ajuste_lluvia.drop(idx_fecha_existente)

    # Crear una nueva fila con la fecha siguiente
    nueva_fila = {col: "" for col in tabla_ajuste_lluvia.columns}
    nueva_fila["FECHA"] = fecha_siguiente_str

    for estacion in tabla_ajuste_lluvia.columns[1:]:
        # Obtener el valor de climatología mensual de lluvia observada para la estación y el mes correspondiente
        valor_min = tablas['min'].loc[tablas['min']["FECHA"] == fecha_siguiente.month, estacion].values[0]
        valor_max = tablas['max'].loc[tablas['max']["FECHA"] == fecha_siguiente.month, estacion].values[0]

        # Verificar si existen filas que coincidan con la estación en data_modelo
        filas_estacion = data_modelo.loc[data_modelo["Estacion"].astype(str).str.strip() == estacion.strip()]

        # Verificar que la fila para la 'estación' en lluvia pronosticada no esté vacía
        if not filas_estacion.empty:
            # Obtener el valor de lluvia pronosticada para la estación y el mes correspondiente
            valor_modelo = filas_estacion["Valor"].iloc[0]

            # Ajustar la lluvia pronosticada con los limites de la climatología mensual de lluvia observada
            if valor_modelo >= valor_max:
                nueva_fila[estacion] = f"{valor_max}({valor_modelo})"
                data_modelo.loc[filas_estacion.index, "Valor"] = valor_max
            elif valor_modelo < 0:
                nueva_fila[estacion] = f"{valor_min}({valor_modelo})"
                data_modelo.loc[filas_estacion.index, "Valor"] = valor_min
            else:
                nueva_fila[estacion] = "-" # Si el valor está dentro de los limites, asignar "-"
        else:
            # Si no hay valor para la estación en data_modelo, asignar "x"
            nueva_fila[estacion] = "x"

    # Agregar la nueva fila a la tabla de ajuste
    tabla_ajuste_lluvia = tabla_ajuste_lluvia.append(nueva_fila, ignore_index=True)

    # Guardar la tabla ajustada y data_modelo en un archivo CSV y TXT
    tabla_ajuste_lluvia.to_csv(f"{carpeta}/tabla_ajuste_lluvia.csv", index=False)
    tabla_ajuste_lluvia.to_csv(os.path.join(carpeta, "tabla_ajuste_lluvia.txt"), index=False)
    
    data_modelo.to_csv(os.path.join(carpeta, f"{file_date}.lluvia.txt"), index=False)
    data_modelo.to_csv(f"{carpeta}/{file_date}.lluvia.csv", index=False)

    return tabla_ajuste_lluvia

### Crear y Actualizar Tabla de Datos Ajustados.

La función `crear_tabla_si_no_existe` se encarga de crear una tabla en una carpeta especificada si esta no existe. La función recibe los siguientes parámetros: `carpeta`, `nombre_tabla`, `enlace_base` y`archivo_excel`

La función comienza comprobando si el archivo en la carpeta especificada existe. Si no existe, la función la crea.

Luego, se construye la ruta completa del archivo que contendrá la tabla a partir de la carpeta y el nombre de la tabla.

A continuación, se verifica si el archivo especificado ya existe. Si el archivo no existe, se procede a crear la tabla vacía. Primero, se define una fecha de inicio (en este caso, 9 de mayo de 2023) y se descargan y procesan los datos de precipitación desde el servidor utilizando la función `descargar_datos_desde_servidor`, cargar archivos de la climatología observada con la función `cargar_tablas_desde_excel` y ajustar los datos con la función`procesar_datos_all`.

Si el archivo ya existe, se carga la tabla desde el archivo y se procesan los datos del siguiente día utilizando las funciones `obtener_datos_un_dia`, cargar archivos de la climatología observada con la función `cargar_tablas_desde_excel` y ajustar los datos con la función `procesar_datos_un_dia`.

La función no devuelve ningún valor (`None`), ya que su propósito es crear o actualizar la tabla y guardar los datos procesados en el archivo correspondiente.

In [7]:
def crear_tabla_si_no_existe(carpeta, nombre_tabla, enlace_base, archivo_excel, hojas_excel):
    """
    Verifica si la tabla de ajuste de lluvia existe y corre las demás funciones dependiendo de esto.
    
    Parameters:
        carpeta (str): Ruta de la carpeta donde se guardará la tabla.
        nombre_tabla (str): Nombre de la tabla a crear.
        enlace_base (str): Enlace base para descargar los datos desde el servidor.
        archivo_excel (str): Ruta del archivo Excel con las tablas.
        hojas_excel (list): Lista de nombres de las hojas de Excel a cargar.

    Returns:
        None
    """
    # Comprobar si la carpeta existe, si no, la crea
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Ruta completa del archivo que contendrá la tabla de ajuste
    ruta_archivo = os.path.join(carpeta, f"{nombre_tabla}.csv")

    # Si el archivo no existe, se crea la tabla vacía y se descargan y procesan los datos
    if not os.path.exists(ruta_archivo):
        #Asignar rango de fechas para cargar archivos de lluvia del modelo
        fecha_inicio = datetime(2023, 5, 9).date()
        fecha_fin = datetime.now().date() + timedelta(days=1)
        #Función que carga todos los archivos de lluvia pronosticada desde el servidor
        data_modelo, file_date = descargar_datos_desde_servidor(enlace_base, fecha_inicio, fecha_fin)
        #Cargar las tablas de la climatología mensual de la lluvia observada
        tablas = cargar_tablas_desde_excel(archivo_excel, hojas_excel)
        #Ajustar los datos de lluvia pronosticada con las tablas de la climatología mensual de lluvia observada
        procesar_datos_all(data_modelo, file_date, tablas, carpeta)
    else:
        # Si el archivo existe, cargar la tabla y procesar el día siguiente
        tabla_ajuste_lluvia = pd.read_csv(ruta_archivo)
        #Función que carga el archivo correspondiente a la lluvia pronosticada del día siguiente desde el servidor
        data_modelo, file_date = obtener_datos_un_dia(enlace_base)
        #Cargar las tablas de la climatología mensual de la lluvia observada
        tablas = cargar_tablas_desde_excel(archivo_excel, hojas_excel)
        #Ajustar los datos de lluvia pronosticada con las tablas de la climatología mensual de lluvia observada
        procesar_datos_un_dia(tabla_ajuste_lluvia, data_modelo, file_date, tablas, carpeta)

# Definición de la ruta de la carpeta donde se almacenarán los datos.
carpeta = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Mod_ajuste_w_Clim/Datos/Lluvia"
# Nombre de la tabla de ajuste a crear o actualizar.
nombre_tabla = "tabla_ajuste_lluvia"
# Enlace base al servidor donde se encuentran los datos de lluvia pronosticada.
enlace_base = "https://wrf1-5.imn.ac.cr/modelo/backup/TESIS/"
# Ruta y nombres de las hojas del archivo Excel que contiene la climatología mensual de lluvia observada.
archivo_excel = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Observado/Lluvia/Climatologia_Mensual_Observada_Lluvia.xlsx'
hojas_excel = ['min', 'max']
# Llamada al método 'crear_tabla_si_no_existe' con los parámetros definidos.
crear_tabla_si_no_existe(carpeta, nombre_tabla, enlace_base, archivo_excel, hojas_excel)

C:\Users\arias\AppData\Local\Temp\ipykernel_23584\2029745463.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabla_ajuste_lluvia = tabla_ajuste_lluvia.append(nueva_fila, ignore_index=True)
